In [9]:
#data_prep.py
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def load_data(path):
    # adjust filename if different
    df = pd.read_csv(path, parse_dates=['Timestamp'])
    df = df.sort_values('Timestamp').reset_index(drop=True)
    return df

def basic_clean(df):
    # lower-case column names
    df.columns = [c.strip() for c in df.columns]
    # ensure timestamp
    if 'Timestamp' not in df.columns:
        raise ValueError("Timestamp column missing")
    df = df.set_index('Timestamp').asfreq('h')  # assume hourly data; fills missing index
    return df

def feature_engineer(df):
    df = df.copy()

    # Try to identify the numeric target column
    possible_targets = [c for c in df.columns if 'energy' in c.lower() or 'usage' in c.lower()]
    if not possible_targets:
        raise ValueError("No obvious target column found. Please specify one manually.")
    target_col = possible_targets[0]

    # Ensure target column is numeric
    df[target_col] = pd.to_numeric(df[target_col], errors='coerce')

    # Drop rows where target is missing or invalid
    df = df.dropna(subset=[target_col])

    # Time-based features
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

    # Optional weather features
    if 'Temperature' in df.columns:
        df['temp'] = pd.to_numeric(df['Temperature'], errors='coerce')
    if 'Humidity' in df.columns:
        df['humidity'] = pd.to_numeric(df['Humidity'], errors='coerce')

    # Lag features (use safe numeric ops)
    for lag in [1, 24, 168]:
        df[f'lag_{lag}'] = df[target_col].shift(lag)

    # Rolling statistics (safe numeric aggregation)
    df['roll_24_mean'] = df[target_col].shift(1).rolling(window=24, min_periods=1).mean()
    df['roll_24_std'] = df[target_col].shift(1).rolling(window=24, min_periods=1).std()

    # One-hot for simple categories
    df['hour_str'] = df['hour'].astype(str)

    # Final cleanup
    df = df.dropna(subset=[target_col, 'lag_1'])

    print(f"✅ Using target column: {target_col}")
    print(f"✅ Data after feature engineering: {df.shape}")

    return df, target_col

def get_train_val(df, val_hours=24*7):
    # time-based split: last `val_hours` hours as validation
    df = df.sort_index()
    val_start = df.index.max() - pd.Timedelta(hours=val_hours)
    train = df[df.index <= val_start].copy()
    val = df[df.index > val_start].copy()
    return train, val

In [10]:
DATA_PATH = "Energy_consumption.csv"   # update with your dataset path

# 1. Load and prepare data
df = load_data(DATA_PATH)
df = basic_clean(df)
df, target_col = feature_engineer(df)


✅ Using target column: HVACUsage
✅ Data after feature engineering: (0, 22)
